In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
if(os.path.split(os.getcwd())[1] == "classify-to-learn"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = False

Current Working Directory: progettoVIPM


In [3]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset
from utils.v2 import ExtendedEncoder, Encoder

In [4]:
from utils.loadersAndEnums import datasets, ImageDataset
from torch.utils.data import DataLoader, ConcatDataset, random_split

trainU = ImageDataset(dataset=datasets.TRAINING_UNLABELED,network_input_size=224, cuda=cuda)
unlabeled = DataLoader(trainU,64,True)


In [5]:

device = torch.device('cpu')
#PATH = "Storage\models\AutoEncoderModelV2.pth"
PATH = 'Storage\models\\FineTuned_ResNet50_80_Train.pth'
model = models.resnet50()
model.fc = nn.Linear(model.fc.in_features, 251)
if cuda:
    model.load_state_dict(torch.load(PATH))
else:
    model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu') ) )
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
all_preds = []
all_out = []
model.eval()
with torch.no_grad():
    i=0
    for inputs,labels in tqdm(unlabeled):
        if i%50==0:
            np.array(all_preds).tofile('all_preds_parziale.dat')
            np.array(all_out).tofile('all_out_parziale.dat')
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        print('outputs: ', outputs)
        print('outputs: ', outputs.shape)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_out.extend(outputs.cpu().numpy())
        i+=1
        
all_preds = np.array(all_preds)
all_out = np.array(all_out)


In [11]:
all_preds.tofile('all_preds_from_resnet_5080.dat')

In [12]:
all_out.tofile('all_out_from_resnet_5080.dat')

In [7]:
all_outfromfile = np.fromfile('all_out_from_resnet_5080.dat', dtype=np.float32)
print('all_outfromfile.shape: ', all_outfromfile.shape)
print('all_outfromfile: ', all_outfromfile)

all_predsfromfile = np.fromfile('all_preds_from_resnet_5080.dat',dtype=np.int64)
print('all_predsfromfile: ', all_predsfromfile)

all_outfromfile.shape:  (28477205,)
all_outfromfile:  [-3.6530356 -3.1039214 -4.272348  ... -2.616568  -2.5592525 -3.5694938]
all_predsfromfile:  [ 94 116 165 ... 195  13 160]


In [8]:
threshold=0.04
probs=[]
confident_enough=[]
confident_enough_classes=[]
for i in range(0, len(all_outfromfile), 251):
    out_proba_of_one_pic=all_outfromfile[i:i+251]
    most_proba = np.argmax(out_proba_of_one_pic)
    prob = all_outfromfile[i+most_proba]
    prob_norm = prob/np.mean(out_proba_of_one_pic) #to reduce the variance of the prob of the most probable class between images
    if prob_norm >= threshold:
        confident_enough.append(int(i/251))
        confident_enough_classes.append(most_proba)

print(len(confident_enough))

171


In [9]:
#Il seguente codice serve per generare un file di annotazione che contenga alcuni unlabeled con la classe predetta dal classificatore
train_set_unlabelled = np.loadtxt(datasets.TRAINING_UNLABELED.value[0], delimiter=",", dtype=np.str_)
print("Total original unlabelled examples: {}".format(train_set_unlabelled.shape[0]))

# Select the most confident images predicted's rows
selected_unlabelled = train_set_unlabelled[confident_enough, :]
selected_unlabelled[:, 1]= confident_enough_classes #change the values in the second column to be the predicted classes
print("Shape of selected_unlabelled training set:{}".format(selected_unlabelled.shape))

filename = datasets.TRAINING_UNLABELED_MOST_CONFIDENTLY_CLASSIFIED.value[0]
np.savetxt(filename, selected_unlabelled,  delimiter = ",", fmt='%s')
print("filename of selected_unlabelled saved is: {}".format(filename))

Total original unlabelled examples: 113455
Shape of selected_unlabelled training set:(171, 2)
filename of selected_unlabelled saved is: train_cbl.csv


In [10]:
import csv
translate_label = []

# Read the CSV file
with open('disambiguation.csv', mode='r') as file:
    reader = csv.reader(file)
    for row in reader:
        label = row[1]
        # Ensure the list is large enough to hold the current index
        
        translate_label.append(label)

print(translate_label)


['macaron', 'bignet', 'cruller', 'vongole_clams', 'triangoli_fritti', 'tiramisu', 'mexican_tostadas', 'lamb_moussaka', 'ravioli_cinesi', 'sushi_sashimi', 'knishes', 'crocchette', 'couscous_verdure', 'porridge', 'lattuga_kimchi', 'alga_wakame', 'spaghetti_squash', 'pasta_rigatoni', 'tartare_di_carne', 'cannolo_siciliano', 'fegato_oca', 'cupcakes', 'carne_ossobuco', 'tagliatelle_con_frutti_di_mare', 'canadian_poutine_w_mushroom_gravy', 'ramen', 'panino_pulled_pork', 'ramen_con_uovo_in_mezzo', 'chicken_kiev', 'torta_mele_stile_crostata', 'risotto', 'torta_stile_torrone_canditi', 'maccheroni_con_carne', 'gluten_free_haggis', 'uova_strapazzate_colazione', 'frittata', 'shrimp_scampi', 'sushi_maki', 'orzo_bolognese', 'amish_apple_fritters', 'nachos', 'beef_stroganoff', 'beef_wellington', 'involtino_primavera', 'dolce_rotondo_con_buco', 'gamberoni', 'souffle', 'maiale_cinese', 'muffin', 'uovo_sodo_con_mousse', 'escargots', 'tramezzino_pollo_club_sandwich', 'torta_di_carote', 'falafell', 'farfa

In [15]:
test20DL = ImageDataset(dataset=datasets.VALIDATION_LABELED_20,network_input_size=224, cuda=cuda)
test20DL = DataLoader(test20DL,64,True)

#### performance del modello "base" (di nostro ha visto l'80% dei dati e basta):

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test20DL:
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total} %')

Accuracy: 32.669322709163346%


### uso le immagini dell'unlabeled set del quale resnet è abbastanza confident, per addestrarlo ulteriormente:

In [17]:
mcc = ImageDataset(dataset=datasets.TRAINING_UNLABELED_MOST_CONFIDENTLY_CLASSIFIED,network_input_size=224, cuda=cuda)
mcc = DataLoader(mcc,64,True)

In [21]:
num_epochs = 10

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

criterion = torch.nn.CrossEntropyLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    for inputs, labels in mcc:
        labels = [int(label) for label in labels]
        inputs = inputs.to(device) 
        labels = torch.tensor(labels, dtype=torch.long).to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(mcc)}')

Epoch 1/10, Loss: 5.664835646487119
Epoch 2/10, Loss: 4.528047784030089
Epoch 3/10, Loss: 4.40519306534215
Epoch 4/10, Loss: 4.314201609433046
Epoch 5/10, Loss: 4.225366486443414
Epoch 6/10, Loss: 4.338887576471295
Epoch 7/10, Loss: 4.293132589574446
Epoch 8/10, Loss: 4.232532307418467
Epoch 9/10, Loss: 4.209317850090607
Epoch 10/10, Loss: 4.192587769519516


In [23]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test20DL:
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total} %')

Accuracy: 0.398406374501992 %
